<a href="https://colab.research.google.com/github/chacha715/ESAA_OB/blob/main/p1_%EA%B2%A9%EC%9E%90%EA%B3%B5%EA%B0%84_%EC%B7%A8%ED%95%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/ESAA/23-1/DATA/물류 유통량 예측 경진대회/") # working directory 설정

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.rc('font', family='NanumBarunGothic') #matplotlib 한글 깨짐 방지

import warnings
warnings.filterwarnings(action='ignore')

In [14]:
train = pd.read_csv("train.csv", encoding='utf-8', index_col=0)
test = pd.read_csv('test.csv', encoding='utf-8', index_col=0)

submission = pd.read_csv('sample_submission.csv')

In [15]:
train.head()

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
index,,,,
0,5011000595017300,2871000192069300,음반,3
1,4148000690043300,5011000264024400,문화컨텐츠,3
2,5011000078068400,1120000007005400,농산물,3
3,4127100048006400,5011000587019400,기타식품,7
4,5011000078068400,2823700010076300,농산물,3


In [16]:
test.head()

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리
index,,,
0,4167000577042200,5011000435014100,선케어
1,1156000009012200,5011000172034400,구강위생용품
2,4122000363057300,5011000361097300,캠핑
3,5011000436041400,2826000084036400,아웃도어가구
4,4150000241065200,5011000169044300,분유/이유식/아기간식


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31684 entries, 0 to 31683
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   송하인_격자공간고유번호  31684 non-null  int64 
 1   수하인_격자공간고유번호  31684 non-null  int64 
 2   물품_카테고리       31684 non-null  object
 3   운송장_건수        31684 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.2+ MB


- null 데이터 없이 잘 들어가 있음을 알 수 있다. 

In [18]:
train.isnull().sum()

송하인_격자공간고유번호    0
수하인_격자공간고유번호    0
물품_카테고리         0
운송장_건수          0
dtype: int64

In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7920 entries, 0 to 7919
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   송하인_격자공간고유번호  7920 non-null   int64 
 1   수하인_격자공간고유번호  7920 non-null   int64 
 2   물품_카테고리       7920 non-null   object
dtypes: int64(2), object(1)
memory usage: 247.5+ KB


- null 데이터 없이 잘 들어가 있음을 알 수 있다. 

# 외부 데이터 관찰



-   앞 5자리가 시군구코드, 251 종류

In [20]:
info_1000 = pd.read_csv("TC_NU_SPG_1000_METER.csv", encoding='utf-8')

In [21]:
info_1000.nunique()

격자공간고유번호    108425
격자공간명       107555
시군구코드          251
시군구명           228
dtype: int64

In [35]:
info_1000.head()

,격자공간고유번호,격자공간명,시군구코드,시군구명
0,4794000001000000,사사0055,47940,울릉군
1,4794000002000000,사사0155,47940,울릉군
2,4794000003000000,사사0255,47940,울릉군
3,4794000004000000,사사0254,47940,울릉군
4,4794000005000000,바사9554,47940,울릉군


# 격자공간 고유번호 의미
- 시군구코드: 첫 5자 / 동: 이후 5자
- 시: 첫 2자, 군/구: 3-5번째, 동: 6-10번째
- 군/구가 달라도 같은 동이름은 있을 수 있음 --> 숫자는 다르겠지? 
- 혹은 전혀 다른 지역이 같은 동숫자를 사용하고 있을수도 있을까?
- 나머지

- 격자고유번호가 숫자로 되어 있어 슬라이싱 되지 않으므로 문자로 타입을 먼저 바꾼다

In [22]:
train['송하인_격자공간고유번호'] = train['송하인_격자공간고유번호'].astype(str)
train['수하인_격자공간고유번호'] = train['수하인_격자공간고유번호'].astype(str)

In [23]:
train['송하인_시군구코드_5'] = train['송하인_격자공간고유번호'].map(lambda x:x[:5])
train['수하인_시군구코드_5'] = train['수하인_격자공간고유번호'].map(lambda x:x[:5])

In [24]:
train['송하인_동코드_5to10'] = train['송하인_격자공간고유번호'].map(lambda x:x[5:10])
train['수하인_동코드_5to10'] = train['수하인_격자공간고유번호'].map(lambda x:x[5:10])

In [25]:
train.head()

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,송하인_시군구코드_5,수하인_시군구코드_5,송하인_동코드_5to10,수하인_동코드_5to10
index,,,,,,,,
0,5011000595017300,2871000192069300,음반,3,50110,28710,00595,00192
1,4148000690043300,5011000264024400,문화컨텐츠,3,41480,50110,00690,00264
2,5011000078068400,1120000007005400,농산물,3,50110,11200,00078,00007
3,4127100048006400,5011000587019400,기타식품,7,41271,50110,00048,00587
4,5011000078068400,2823700010076300,농산물,3,50110,28237,00078,00010


In [30]:
train['송하인_시군구코드_5'].nunique()

237

In [32]:
train['수하인_시군구코드_5'].nunique()

251

*   train 데이터에서 시군구구코드는 송하인 237종류, 수하인 251종류로 외부 데이터에 모두 포함되어있으리라 예상

In [26]:
test['송하인_격자공간고유번호'] = test['송하인_격자공간고유번호'].astype(str)
test['수하인_격자공간고유번호'] = test['수하인_격자공간고유번호'].astype(str)

In [27]:
test['송하인_시군구코드_5'] = test['송하인_격자공간고유번호'].map(lambda x:x[:5])
test['수하인_시군구코드_5'] = test['수하인_격자공간고유번호'].map(lambda x:x[:5])

In [28]:
test['송하인_동코드_5to10'] = test['송하인_격자공간고유번호'].map(lambda x:x[5:10])
test['수하인_동코드_5to10'] = test['수하인_격자공간고유번호'].map(lambda x:x[5:10])

In [29]:
test.head()

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,송하인_시군구코드_5,수하인_시군구코드_5,송하인_동코드_5to10,수하인_동코드_5to10
index,,,,,,,
0,4167000577042200,5011000435014100,선케어,41670,50110,00577,00435
1,1156000009012200,5011000172034400,구강위생용품,11560,50110,00009,00172
2,4122000363057300,5011000361097300,캠핑,41220,50110,00363,00361
3,5011000436041400,2826000084036400,아웃도어가구,50110,28260,00436,00084
4,4150000241065200,5011000169044300,분유/이유식/아기간식,41500,50110,00241,00169


In [31]:
test['송하인_시군구코드_5'].nunique()

196

In [34]:
test['수하인_시군구코드_5'].nunique()

250

*   test 데이터에서 시군구코드는 송하인 196종류, 수하인 250종류로 외부 데이터에 모두 포함되어있으리라 예상

# 레이블 인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(train['물품_카테고리'])

train['물품_카테고리'] = encoder.transform(train['물품_카테고리'])
test['물품_카테고리'] = encoder.transform(test['물품_카테고리'])

## 모델링

In [ ]:
from lightgbm import LGBMRegressor

train_X = train.drop('운송장_건수',axis = 1)
train_Y = train['운송장_건수']

#모델 정의
model = LGBMRegressor()

In [ ]:
# 모델 학습
model.fit(train_X,train_Y)

LGBMRegressor()

In [ ]:
# test 데이터 예측
pred = model.predict(test)

## 정답파일 생성

In [ ]:
submission.to_csv('baseline.csv',index = False)

https://yganalyst.github.io/spatial_analysis/spatial_analysis_1/
시각화 툴